<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/SNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install snntorch -q

In [ ]:
!pip uninstall torch torchvision torchaudio

In [ ]:
!pip install torch -q

In [ ]:
!pip install torchvision -q

In [5]:
import torch
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen
import torch.nn as nn
from torchvision import datasets, transforms
import torchvision
import snntorch

In [6]:
print(torch.__version__)
print(snntorch.__version__)
print(torchvision.__version__)

2.5.1+cu124
0.9.1
0.20.1+cu124


In [7]:
!pip install --upgrade snntorch -q

In [8]:
print(torch.__version__)
print(snntorch.__version__)
print(torchvision.__version__)

2.5.1+cu124
0.9.1
0.20.1+cu124


In [24]:
!nvidia-smi

Sat Nov  2 19:50:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   76C    P0              34W /  72W |   1263MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [23]:
import torch
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen
import torch.nn as nn
from torchvision import datasets, transforms

# Check if a CUDA-enabled GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use CUDA GPU
    print("Using GPU for training.")
else:
    device = torch.device("cpu")  # Use CPU
    print("GPU not available, using CPU.")



# Define the network architecture

class Net(torch.nn.Module):
    def __init__(self):
       super().__init__()

        # Input layer
       self.fc1 = torch.nn.Linear(784, 1000)

        # Spiking neuron layer
       self.lif1 = snn.Leaky(beta=0.9)

        # Output layer
       self.fc2 = torch.nn.Linear(1000, 10)

    def forward(self, x):
        # Convert input to spikes
        spk_in = spikegen.rate(x, num_steps=25)

        # Ensure spk_in is a 3D tensor
        if len(spk_in.shape) == 2:
            spk_in = spk_in.unsqueeze(0)

        # Pass through the network
        cur1 = self.fc1(spk_in)
        spk1 = self.lif1(cur1)
        spk1 = spk1[0]  # Select the spike train

        cur2 = self.fc2(spk1)
        spk2 = self.lif1(cur2)
        spk2 = spk2[0]  # Select the spike train

        return spk2

# Instantiate the network and move it to the device
net = Net().to(device)

# Define the optimizer and loss function

optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

# Load MNIST dataset
train_dataset = datasets.MNIST(
    root='./data', train=True, download=True, transform=transforms.ToTensor()
)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=64, shuffle=True
)

test_dataset = datasets.MNIST(
    root='./data', train=False, download=True, transform=transforms.ToTensor()
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1000, shuffle=False
)

# Training loop
num_epochs = 5  # Increased number of epochs
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move images and labels to the device
        images = images.reshape(-1, 784).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = net(images)

        # Convert output spikes to firing rates
        output_rates = torch.mean(outputs, dim=0)

        # Calculate loss
        loss = loss_fn(output_rates, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(
                f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}'
            )

# Model evaluation
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        # Move images and labels to the device
        images = images.reshape(-1, 784).to(device)
        labels = labels.to(device)

        outputs = net(images)
        output_rates = torch.mean(outputs, dim=0)
        _, predicted = torch.max(output_rates.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('\n')
print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Using GPU for training.
Epoch [1/5], Step [100/938], Loss: 1.5484
Epoch [1/5], Step [200/938], Loss: 1.5223
Epoch [1/5], Step [300/938], Loss: 1.5059
Epoch [1/5], Step [400/938], Loss: 1.5372
Epoch [1/5], Step [500/938], Loss: 1.4833
Epoch [1/5], Step [600/938], Loss: 1.5143
Epoch [1/5], Step [700/938], Loss: 1.5014
Epoch [1/5], Step [800/938], Loss: 1.5108
Epoch [1/5], Step [900/938], Loss: 1.5005
Epoch [2/5], Step [100/938], Loss: 1.5243
Epoch [2/5], Step [200/938], Loss: 1.5172
Epoch [2/5], Step [300/938], Loss: 1.5151
Epoch [2/5], Step [400/938], Loss: 1.4749
Epoch [2/5], Step [500/938], Loss: 1.4717
Epoch [2/5], Step [600/938], Loss: 1.4796
Epoch [2/5], Step [700/938], Loss: 1.4904
Epoch [2/5], Step [800/938], Loss: 1.4982
Epoch [2/5], Step [900/938], Loss: 1.5017
Epoch [3/5], Step [100/938], Loss: 1.4712
Epoch [3/5], Step [200/938], Loss: 1.4776
Epoch [3/5], Step [300/938], Loss: 1.4771
Epoch [3/5], Step [400/938], Loss: 1.4855
Epoch [3/5], Step [500/938], Loss: 1.4714
Epoch [3/5